In [1]:
import sys
sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# all envs

from environment.deepqlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv
from utils.reader import get_yaml_path, read_file

In [4]:
server_address = "localhost:50060"
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name)
env.connect_to_client()

2025-11-01 16:46:06,912 — INFO — ✓ Connected to localhost:50060



In [5]:
config_path = get_yaml_path("resources", "configurations", "obstacle-avoidance.yml")
config = read_file(config_path)
# print(config)

In [6]:
env.init(config)

2025-11-01 16:46:06,932 — INFO — ✓ Initialization successful


In [7]:
from agent.scala_dqagent import DQAgent
from training.dqnetwork import DQNetwork

2025-11-01 16:46:07.193053: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
neuron_count_per_hidden_layer = [64, 32]

In [9]:
agent1 = DQAgent(
    env,
    agent_id="00000000-0000-0000-0000-000000000001",
    action_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
    target_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
)
agents = [agent1]

I0000 00:00:1762011973.448804   20843 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3822 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:1d:00.0, compute capability: 6.1


In [13]:
path = get_yaml_path("src", "scripts", "checkpoints", "oa_ep1200")
for agent in agents:
    agent.load(path)

/nix/store/pwrhqv9fy9lbybjz9c1w283z234lzksr-python3-3.13.8-env/lib/python3.13/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
import time

from training.multi_agent_dqlearning import DQLearning

trainer = DQLearning(
    env,
    agents,
)

In [15]:
trainer.play_with_pygame(episodes=5, fps=60)

2025-11-01 17:28:09,077 — INFO — Episode 1/5 - Reward: 1105.0870195294947
2025-11-01 17:28:42,635 — INFO — Episode 2/5 - Reward: 1105.0870195294947
2025-11-01 17:28:48,178 — INFO — Episode 3/5 - Reward: 152.35867744281967
2025-11-01 17:28:48,179 — INFO — Episode 4/5 - Reward: 0
2025-11-01 17:28:48,181 — INFO — Episode 5/5 - Reward: 0
